<a href="https://colab.research.google.com/github/Gaveloshpom/GoogleColabs/blob/main/Lab4_622_Skytsko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>МІНІСТЕРСТВО ОСВІТИ І НАУКИ УКРАЇНИ</center>

<center>Національний аерокосмічний університет ім. М. Є. Жуковського  
«Харківський авіаційний інститут»</center>

<center>факультет програмної інженерії та бізнесу</center>
<center>кафедра інженерії програмного забезпечення</center>

---

<center>Лабораторна робота №4</center>

<center>з дисципліни «Штучний інтелект та бази знань»</center>

---

**Виконав:** студент 2 курсу групи **622П**  
напряму підготовки (спеціальності) - **121 інженерія програмного забезпечення**  
**Скицко Р. С.**

---

<center>Харків — 2025</center>

# **Завдання 1:** Дослідити вплив значень кількості епох та швидкості навчання на точність передбачень

## 1.Проаналізуйте запропоновану реалізацію простої нейромережі на Python з використанням бібліотеки NumPy та перенесіть у власний notebook

### **Імпорт необхідних бібліотек**

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import sys

### **Код**

In [ ]:
np.random.seed(42)

class SimpleNeuralNetwork(object):

    def __init__(self, learning_rate=0.1): # конструктор
      self.weights_0_1 = np.random.normal(0.0, 2 ** -0.5, (2, 3)) # радномні ваги від вхідного шару до прихованого, розмірність 2 на 3, в прихованому шарі 2 нейрони і кожен має 3 значення вагів
      self.weights_1_2 = np.random.normal(0.0, 1, (1, 2)) # радномні ваги від прихованого шару до вихідного, 1 нейрон та 2 значення (зв'язок з 2 нейронами прихованого шару)
      self.sigmoid_mapper = np.vectorize(self.sigmoid) # проходимо по вектору і до кожного елементу приміняємо сігмоідну функцію
      self.learning_rate = np.array([learning_rate]) # зберігає значенн яlearning rate

    def sigmoid(self, x):
      return 1 / (1 + np.exp(-x)) # формула сігмоіду

    def predict(self, inputs): # метод прямого розповсюдження для передбачень, inputs = 1 або 0 та має розмірність 3
      inputs_1 = np.dot(self.weights_0_1, inputs) # ваги від вхідного шару до прихованого помнажаються на inputs, отримуємо inputs до прихованого шару
      outputs_1 = self.sigmoid_mapper(inputs_1) # пропускаємо через активаційну функцію (сігмоід)

      inputs_2 = np.dot(self.weights_1_2, outputs_1) # помножаємо ваги від прихованого шару до останнього шару
      outputs_2 = self.sigmoid_mapper(inputs_2) # пропускаємо через активаційну функцію (сігмоід)
      return outputs_2 # результат (0 або 1)

    def train(self, inputs, expected_predict): # звортнє розповсюдження помилки
        inputs_1 = np.dot(self.weights_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)

        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        actual_predict = outputs_2[0] # зберігаємо поточний результат

        error_layer_2 = np.array([actual_predict - expected_predict]) # розраховуємо помилку
        gradient_layer_2 = actual_predict * (1 - actual_predict) # розраховуємо градієнт (похідну від сігмоідальної функції)
        weights_delta_layer_2 = error_layer_2 * gradient_layer_2 # delta вагів
        self.weights_1_2 -= (np.dot(weights_delta_layer_2, outputs_1.reshape(1, len(outputs_1)))) * self.learning_rate # корегуємо ваги

        error_layer_1 = weights_delta_layer_2 * self.weights_1_2 # перемножуємо delta вагів на ваги прихованого шару
        gradient_layer_1 = outputs_1 * (1 - outputs_1) # розраховуємо градієнт (похідну від сігмоідальної функції)
        weights_delta_layer_1 = error_layer_1 * gradient_layer_2
        self.weights_0_1 -= np.dot(inputs.reshape(len(inputs), 1), weights_delta_layer_1).T * self.learning_rate # корегуємо ваги

def MSE(y, Y):
    return np.mean((y-Y)**2)


train = [
    ([0, 0, 0], 0),
    ([0, 0, 1], 1),
    ([0, 1, 0], 0),
    ([0, 1, 1], 0),
    ([1, 0, 0], 1),
    ([1, 0, 1], 1),
    ([1, 1, 0], 0),
    ([1, 1, 1], 0),
]

In [ ]:
epochs = 200 # задаємо кількість епох
learning_rate = 0.01 # задаємо швидкість навчання

network = SimpleNeuralNetwork(learning_rate=learning_rate) # створюємор мережу

for e in range(epochs):
    inputs_ = []
    correct_predictions = []
    for input_stat, correct_predict in train:
        network.train(np.array(input_stat), correct_predict) # передаємо вхідні дані та очікувані
        inputs_.append(np.array(input_stat))
        correct_predictions.append(np.array(correct_predict))

    train_loss = MSE(network.predict(np.array(inputs_).T), np.array(correct_predictions)) # розраховуємо MSE
    sys.stdout.write("\rПрогрес: {}%, Тренувальна втрата: {}".format(str(100 * e/float(epochs))[:4], str(train_loss)[:5]))

Прогрес: 99.5%, Тренувальна втрата: 0.212

In [ ]:
for input_stat, correct_predict in train: # виконуємо прогнозування
    print("Для вхідних даних: {} передбачення: {}, очікуваний результат: {}".format(
        str(input_stat),
        str(network.predict(np.array(input_stat)) > .5),
        str(correct_predict == 1)))

Для вхідних даних: [0, 0, 0] передбачення: [False], очікуваний результат: False
Для вхідних даних: [0, 0, 1] передбачення: [False], очікуваний результат: True
Для вхідних даних: [0, 1, 0] передбачення: [False], очікуваний результат: False
Для вхідних даних: [0, 1, 1] передбачення: [False], очікуваний результат: False
Для вхідних даних: [1, 0, 0] передбачення: [False], очікуваний результат: True
Для вхідних даних: [1, 0, 1] передбачення: [False], очікуваний результат: True
Для вхідних даних: [1, 1, 0] передбачення: [False], очікуваний результат: False
Для вхідних даних: [1, 1, 1] передбачення: [False], очікуваний результат: False


In [ ]:
for input_stat, correct_predict in train: # виконуємо прогнозування
    print("Для вхідних даних: {} передбачення: {}, очікуваний результат: {}".format(
        str(input_stat),
        str(network.predict(np.array(input_stat))),
        str(correct_predict == 1)))

Для вхідних даних: [0, 0, 0] передбачення: [0.25369793], очікуваний результат: False
Для вхідних даних: [0, 0, 1] передбачення: [0.32371759], очікуваний результат: True
Для вхідних даних: [0, 1, 0] передбачення: [0.21182392], очікуваний результат: False
Для вхідних даних: [0, 1, 1] передбачення: [0.27603489], очікуваний результат: False
Для вхідних даних: [1, 0, 0] передбачення: [0.30103863], очікуваний результат: True
Для вхідних даних: [1, 0, 1] передбачення: [0.36839129], очікуваний результат: True
Для вхідних даних: [1, 1, 0] передбачення: [0.25387103], очікуваний результат: False
Для вхідних даних: [1, 1, 1] передбачення: [0.32235974], очікуваний результат: False


## 2.Підберіть значення кількості епох (epochs) та швидкості навчання (learning_rate) при яких модель виконує передбачення, що відповідають очікуваному результату, а також відкорегуйте значення втрат до 0.00*.

In [ ]:
epochs = 5000 # задаємо кількість епох
learning_rate = 0.05 # задаємо швидкість навчання

network = SimpleNeuralNetwork(learning_rate=learning_rate) # створюємор мережу

for e in range(epochs):
    inputs_ = []
    correct_predictions = []
    for input_stat, correct_predict in train:
        network.train(np.array(input_stat), correct_predict) # передаємо вхідні дані та очікувані
        inputs_.append(np.array(input_stat))
        correct_predictions.append(np.array(correct_predict))

    train_loss = MSE(network.predict(np.array(inputs_).T), np.array(correct_predictions)) # розраховуємо MSE
    sys.stdout.write("\rПрогрес: {}%, Тренувальна втрата: {}".format(str(100 * e/float(epochs))[:4], str(train_loss)[:5]))

Прогрес: 99.9%, Тренувальна втрата: 0.004

In [ ]:
for input_stat, correct_predict in train: # виконуємо прогнозування
    print("Для вхідних даних: {} передбачення: {}, очікуваний результат: {}".format(
        str(input_stat),
        str(network.predict(np.array(input_stat)) > .5),
        str(correct_predict == 1)))

Для вхідних даних: [0, 0, 0] передбачення: [False], очікуваний результат: False
Для вхідних даних: [0, 0, 1] передбачення: [ True], очікуваний результат: True
Для вхідних даних: [0, 1, 0] передбачення: [False], очікуваний результат: False
Для вхідних даних: [0, 1, 1] передбачення: [False], очікуваний результат: False
Для вхідних даних: [1, 0, 0] передбачення: [ True], очікуваний результат: True
Для вхідних даних: [1, 0, 1] передбачення: [ True], очікуваний результат: True
Для вхідних даних: [1, 1, 0] передбачення: [False], очікуваний результат: False
Для вхідних даних: [1, 1, 1] передбачення: [False], очікуваний результат: False


In [ ]:
for input_stat, correct_predict in train: # виконуємо прогнозування
    print("Для вхідних даних: {} передбачення: {}, очікуваний результат: {}".format(
        str(input_stat),
        str(network.predict(np.array(input_stat))),
        str(correct_predict == 1)))

Для вхідних даних: [0, 0, 0] передбачення: [0.12640348], очікуваний результат: False
Для вхідних даних: [0, 0, 1] передбачення: [0.91025496], очікуваний результат: True
Для вхідних даних: [0, 1, 0] передбачення: [0.00059344], очікуваний результат: False
Для вхідних даних: [0, 1, 1] передбачення: [0.00097007], очікуваний результат: False
Для вхідних даних: [1, 0, 0] передбачення: [0.9110804], очікуваний результат: True
Для вхідних даних: [1, 0, 1] передбачення: [0.95328146], очікуваний результат: True
Для вхідних даних: [1, 1, 0] передбачення: [0.00092987], очікуваний результат: False
Для вхідних даних: [1, 1, 1] передбачення: [0.08110165], очікуваний результат: False


## **Висновки**

Виявлено, що при недостатній кількості епох (менше 500) мережа не встигає навчитися і дає хибні передбачення. Занадто велике значення learning_rate призводить до нестабільного навчання. Найкращий результат було досягнуто при epochs = 5000 та learning_rate = 0.05. За цих умов значення втрат наблизилось до 0.005, а передбачення повністю співпадають з очікуваними результатами.

Таким чином, правильний підбір гіперпараметрів (epochs та learning_rate) суттєво впливає на ефективність навчання нейронної мережі.

# **Завдання 2:** Написати просту нейромережу з прихованим шаром на Keras

## 1.Імпортуйте необхідні бібліотеки та класи з Keras.

In [22]:
# 1. Імпортуємо необхідні бібліотеки
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

## 2.Розподіліть список даних train (з першого завдання) на X_train та y_train, де X_train - вхідні значення, y_train - передбачення.

In [23]:
# 2. Підготовка даних
train = [
    ([0, 0, 0], 0),
    ([0, 0, 1], 1),
    ([0, 1, 0], 0),
    ([0, 1, 1], 0),
    ([1, 0, 0], 1),
    ([1, 0, 1], 1),
    ([1, 1, 0], 0),
    ([1, 1, 1], 0),
]

X_train = np.array([x for x, y in train])
y_train = np.array([y for x, y in train])

## 3.Інціалізуйте послідовну модель Sequential(). Послідовні моделі у Keras використовуються для створення моделей шар за шаром, де кожен наступний шар має з'єднання лише з попереднім.

In [27]:
# 3. Ініціалізація моделі
model = Sequential()

## 4.Додайте повнозв'язний (прихований) шар, що має мати 2 нейрони, та очікувати вхідні дані з 3 особливостями. Функцію активації задаємо сігмоідальну.

In [28]:
# 4. Додаємо прихований шар (2 нейрони, вхід з 3-х ознак)
model.add(Dense(2, input_dim=3, activation='sigmoid'))

## 5.Додайте вихідний шар до моделі. Цей шар має мати лише один нейрон, що означає, що модель буде виробляти одне вихідне значення.

In [29]:
# 5. Додаємо вихідний шар (1 нейрон, активація — сигмоїдна)
model.add(Dense(1, activation='sigmoid'))

## 6.Скомпілюйте модель. В якості функції втрат використайте бінарну кросентропію, в якості оптимізатора -- стохастичний градієнтний спуск, а для оцінки якості моделі використайте метріку точності. В якості початкового значення швидкості навчання використайте визначене в завданні 1.

In [30]:
# 6. Компіляція моделі
learning_rate = 0.05
optimizer = SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

## 7.Натренуйте модель, в якості початкового значення кількості епох вкажіть визначене в завданні 1.

In [31]:
# 7. Навчання моделі
epochs = 5000
model.fit(X_train, y_train, epochs=epochs, verbose=0)

## 8.Згенеруйте передбачення.

In [33]:
# 8. Генерація передбачень
predictions = model.predict(X_train)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


## 9.Оскільки вихідний шар моделі використовує сигмоїдну активаційну функцію для бінарної класифікації, результати передбачень є ймовірностями належності до класу '1'. Використання функції np.round(predictions) округлює ці ймовірності до найближчого цілого числа, що перетворить їх на бінарні класи (0 або 1).

In [34]:
# 9. Округлення ймовірностей до класів (0 або 1)
rounded_predictions = np.round(predictions).astype(int)

## 10.Виведіть передбачення та порівняйте з очікуванними резульататми відповідно до таблиці рішень.

In [35]:
# 10. Виведення результатів
for i, (x, expected) in enumerate(train):
    print(f"Вхідні дані: {x} -> Передбачення: {rounded_predictions[i][0]}, Очікуване: {expected}")

Вхідні дані: [0, 0, 0] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [0, 0, 1] -> Передбачення: 1, Очікуване: 1
Вхідні дані: [0, 1, 0] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [0, 1, 1] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [1, 0, 0] -> Передбачення: 1, Очікуване: 1
Вхідні дані: [1, 0, 1] -> Передбачення: 1, Очікуване: 1
Вхідні дані: [1, 1, 0] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [1, 1, 1] -> Передбачення: 0, Очікуване: 0


## 11.Відкорегуйте значення кількості епох та швидкості навчання, якщо передбачення моделі не сходяться з очікуваними результатами.

In [36]:
# 11. Перевірка точності
loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Втрати: {loss:.5f}, Точність: {accuracy*100:.2f}%")

Втрати: 0.03734, Точність: 100.00%


Передбачення моделі повністю сходяться з очікуваними результатами

## Повний код

In [32]:
# 2. Підготовка даних
train = [
    ([0, 0, 0], 0),
    ([0, 0, 1], 1),
    ([0, 1, 0], 0),
    ([0, 1, 1], 0),
    ([1, 0, 0], 1),
    ([1, 0, 1], 1),
    ([1, 1, 0], 0),
    ([1, 1, 1], 0),
]

X_train = np.array([x for x, y in train])
y_train = np.array([y for x, y in train])

# 3. Ініціалізація моделі
model = Sequential()

# 4. Додаємо прихований шар (2 нейрони, вхід з 3-х ознак)
model.add(Dense(2, input_dim=3, activation='sigmoid'))

# 5. Додаємо вихідний шар (1 нейрон, активація — сигмоїдна)
model.add(Dense(1, activation='sigmoid'))

# 6. Компіляція моделі
learning_rate = 0.05  # взяте з Завдання 1
optimizer = SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 7. Навчання моделі
epochs = 5000  # взяте з Завдання 1
model.fit(X_train, y_train, epochs=epochs, verbose=0)

# 8. Генерація передбачень
predictions = model.predict(X_train)

# 9. Округлення ймовірностей до класів (0 або 1)
rounded_predictions = np.round(predictions).astype(int)

# 10. Виведення результатів
for i, (x, expected) in enumerate(train):
    print(f"Вхідні дані: {x} -> Передбачення: {rounded_predictions[i][0]}, Очікуване: {expected}")

# 11. Перевірка точності
loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"\nВтрати: {loss:.5f}, Точність: {accuracy*100:.2f}%")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Вхідні дані: [0, 0, 0] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [0, 0, 1] -> Передбачення: 1, Очікуване: 1
Вхідні дані: [0, 1, 0] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [0, 1, 1] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [1, 0, 0] -> Передбачення: 1, Очікуване: 1
Вхідні дані: [1, 0, 1] -> Передбачення: 1, Очікуване: 1
Вхідні дані: [1, 1, 0] -> Передбачення: 0, Очікуване: 0
Вхідні дані: [1, 1, 1] -> Передбачення: 0, Очікуване: 0

Втрати: 0.03734, Точність: 100.00%


## Висновки

Експериментально підібрано гіперпараметри: 1. кількість епох: 5000, 2. швидкість навчання: 0.05.

Після навчання моделі значення втрат знизилось до 0.03, точність становила 100%, передбачення точно збіглися з очікуваними значеннями.

В процесі стикнулися з доволі довгим навчанням моделі.

Модель успішно впоралась з поставленою задачею.